In [1]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()


In [2]:
print("length of dataset in characters:", len(text))

length of dataset in characters: 1115394


In [3]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
chars = sorted(list(set(text)))
vocab_size =  len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [6]:
print(encode("Hi there"))

[20, 47, 1, 58, 46, 43, 56, 43]


In [7]:
print(decode(encode("Hi there")))

Hi there


In [8]:
import torch

In [9]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [10]:
print(data[:100])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [11]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [12]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [13]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

In [14]:
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [15]:
torch.manual_seed(10)

In [16]:
batch_size=4
block_size=8

In [22]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # randomly sampling batch from dataset
    x = torch.stack([data[i:i+block_size] for i in ix]) # getting the training data 
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # getting the labels 
    return x, y

In [23]:
xb, yb = get_batch('train')
print("inputs:")
print(xb.shape)
print(xb)

inputs:
torch.Size([4, 8])
tensor([[ 1, 58, 46, 43, 43,  6,  1, 49],
        [ 1, 61, 43,  1, 41, 56, 43, 39],
        [43, 41, 58,  0, 32, 47, 50, 50],
        [49, 57,  1, 58, 46, 39, 58,  1]])


In [24]:
yb

tensor([[58, 46, 43, 43,  6,  1, 49, 47],
        [61, 43,  1, 41, 56, 43, 39, 58],
        [41, 58,  0, 32, 47, 50, 50,  1],
        [57,  1, 58, 46, 39, 58,  1, 63]])

In [25]:
for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension 
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

when input is [1] the target: 58
when input is [1, 58] the target: 46
when input is [1, 58, 46] the target: 43
when input is [1, 58, 46, 43] the target: 43
when input is [1, 58, 46, 43, 43] the target: 6
when input is [1, 58, 46, 43, 43, 6] the target: 1
when input is [1, 58, 46, 43, 43, 6, 1] the target: 49
when input is [1, 58, 46, 43, 43, 6, 1, 49] the target: 47
when input is [1] the target: 61
when input is [1, 61] the target: 43
when input is [1, 61, 43] the target: 1
when input is [1, 61, 43, 1] the target: 41
when input is [1, 61, 43, 1, 41] the target: 56
when input is [1, 61, 43, 1, 41, 56] the target: 43
when input is [1, 61, 43, 1, 41, 56, 43] the target: 39
when input is [1, 61, 43, 1, 41, 56, 43, 39] the target: 58
when input is [43] the target: 41
when input is [43, 41] the target: 58
when input is [43, 41, 58] the target: 0
when input is [43, 41, 58, 0] the target: 32
when input is [43, 41, 58, 0, 32] the target: 47
when input is [43, 41, 58, 0, 32, 47] the target: 50
w

## creating BIgram Language model

In [26]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1)

In [67]:
class BigramLanguagemodel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        
        if targets is None:
            loss = None 
        else:
            B, T, C = logits.shape # C is the vocab size here
            #print(B,T, C)
            logits = logits.view(B*T, C) # torch takes in batch first 
            target = targets.view(B*T)
            loss = F.cross_entropy(logits, target)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            
            logits = logits[:, -1, :] # focus only on the last time step
            probs = F.softmax(logits, dim=-1) # (B,C)
            
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            
            idx = torch.cat((idx, idx_next), dim=1) # B, T+1
            
        return idx

In [68]:
m = BigramLanguagemodel(vocab_size)

In [69]:
logits, loss= m(xb, yb)

In [70]:
loss

tensor(4.7728, grad_fn=<NllLossBackward0>)

In [71]:

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


jqPFWUvsHZGALfJaXwHCKlaXVrS&!Xfw$;EWgIzARhoeXRtD.l!hCygQBKEw.Xr BwVEIlSu3nPk&Ygd NydeO&tZjLBq; FWMyW


In [72]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [73]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

4.547042369842529


In [74]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


DIeIEtMoeihR?IRpl
'piuxGWICqEt$dO:jHSP$EAfSph$cOQpAFt o&yVXd3aQc$y'q uM3&&'op'owj a:WgyDu3&qZO:Psr Kk?t&p'3XbznYx?rDHHP'MEic$MfKot!huVXK''owT,jY;oeQ.ZhMWlFaCe'UXyd&tWxml$xUgjlNZYSY'J;L Yh
laYK?Z !p'rJAqQP$KZYj
NBdoP'lCop!dh$,ydJnCmYs,-?Ko bCS'UvKGRLX?VP3F KR&j
XLjEvzRJwYlVnQyKH'qhOd3bnu3h
aXw-.W'dO:KtDH
W; E-.g:UgNzlaCNjpuInobk:!'lItyad$Flqmow'
GUg
d$ xK,!;In l'MEaCOP3WD;&Vgl&otLs
qUOfmyPfOi.HOkoHkaz:hfdLSUSsORclaXEWy
VqZ'PX&YcTGzi
ZLD; K,!CqMxleaXUXGAr'.sec$&Pg
nfPs:K'ii;$UsOsUE?IQ3.QOKR$WCIx3z


## The mathematical trick in self-attention

In [75]:
B, T, C = 4, 8, 2

In [76]:
x = torch.randn(B, T, C)

In [77]:
x.shape

torch.Size([4, 8, 2])

In [78]:
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] #(t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [79]:
x[0]

tensor([[-0.8716,  0.0373],
        [ 0.3508, -2.0232],
        [ 1.4559, -1.2582],
        [-0.5755,  1.0666],
        [ 0.0455,  0.2643],
        [ 1.1503, -0.0786],
        [-1.0749,  1.6004],
        [-0.4880,  1.6751]])

In [80]:
torch.manual_seed(42)
a = torch.ones(3,3)
b = torch.randint(0, 10, (3,2)).float()

In [81]:
b

tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])

In [82]:
c = a@b

In [83]:
c

tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])

In [85]:
torch.dril(a)

AttributeError: module 'torch' has no attribute 'dril'